# Tech Challenge - Fase 2

## 1 - Contexto e Objetivo de Negócio

### 1.1 - Contexto

- O fundo atua em um ambiente de alta volatilidade e forte competição;
- Decisões diárias de alocação em renda variável são fortemente impactadas pela direção do IBOVESPA;
- Hoje, boa parte dessas decisões se baseiam em análises arbitrárias e modelos pontuais, sem um indicador padronizado de tendência de curto prazo do índice.

### 1.2 - Problema de Negócio

- O time de investimentos precisa de um sinal objetivo e recorrente sobre a provável direção do IBOVESPA no pregão seguinte;
- Esse sinal deve ser simples de interpretar (↑ / ↓)/(1 / 0) e fácil de consumir nos dashboards internos já utilizados pelos analistas.

### 1.3 - Objetivo do Projeto

- Desenvolver um modelo preditivo que estime se o fechamento do IBOVESPA amanhã será maior ou menor que o fechamento de hoje;
- Entregar, diariamente, um sinal binário (↑/1 = alta, ↓/0 = baixa) para alimentar os dashboards quantitativos da casa;
- Garantir um nível mínimo de desempenho: acurácia ≥ 75% no último mês de dados (30 dias), respeitando a natureza temporal da série.

### 1.4 - Papel do Modelo

- Não substitui o analista, mas serve como insumo adicional e padronizado na tomada de decisão.
- Servir como base para evoluções futuras, incorporando novos fatores em versões seguintes.

## 2 - Aquisição e Pré-processamento dos Dados

### 2.1 - Fonte dos Dados

- **Origem**: *Investing.com* Brasil – seção “Dados Históricos Ibovespa”, que disponibiliza gratuitamente a série diária do índice.
- **Ativo**: Índice Bovespa (IBOV), principal indicador de desempenho do mercado acionário brasileiro, calculado pela B3.
- **Tipo de dado**: preços de fechamento ajustados, abertura, máxima, mínima, volume negociado e variação percentual diária.

### 2.2 - Período Utilizado

- **13/Dez/2023 a 13/Dez/2025** (últimos 2 anos completos de pregão).
- **Escolha do horizonte**:
  - Longo o suficiente para treinar o modelo com diversidade de cenários;
  - Recente o suficiente para refletir o regime atual de mercado.

### 2.3 - Granularidade e Cobertura

- **Frequência**: dados diários, utilizando apenas pregões em que o IBOV foi negociado (sem finais de semana e feriados).
- Cada linha representa um dia de negociação com:
  - Data do pregão (**Date**);
  - Preço de fechamento (**Close**);
  - Abertura (**Open**);
  - Máxima (**High**) e Mínima (**Low**) do dia;
  - Volume negociado (**Volume**)
  - Variação percentual do dia (**Var%**)

### 2.4 - Importação das Bibliotecas

In [ ]:
# Importação das bibliotecas necessárias

# Manipulação de dados
import pandas as pd
pd.options.display.float_format = '{:f}'.format
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Usa o tema darkgrid do seaborn
sns.set_style('darkgrid')

# Ignorar avisos
import warnings
warnings.filterwarnings('ignore')

### 2.5 - Leitura e Visualização da Base de Dados

In [ ]:
# Leitura da base de dados
df = pd.read_csv("https://raw.githubusercontent.com/JoalysonLima/ML_IBOVESPA/refs/heads/main/Dados%20Hist%C3%B3ricos%20-%20Ibovespa.csv", sep=',')


In [ ]:
# Visualização das primeiras linhas da base de dados
df.head(10)

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,12.12.2025,160.766000,159.189000,161.263000,159.189000,"7,67B","0,99%"
1,11.12.2025,159.189000,159.072000,159.850000,158.098000,"7,02B","0,07%"
2,10.12.2025,159.075000,157.984000,159.691000,157.628000,"8,24B","0,69%"
3,09.12.2025,157.981000,158.187000,158.851000,155.188000,"8,70B","-0,13%"
4,08.12.2025,158.187000,157.369000,159.235000,157.369000,"9,02B","0,52%"
5,05.12.2025,157.369000,164.461000,165.036000,157.007000,"14,53B","-4,31%"
6,04.12.2025,164.456000,161.760000,164.551000,161.759000,"10,59B","1,67%"
7,03.12.2025,161.755000,161.094000,161.963000,161.093000,"8,29B","0,41%"
8,02.12.2025,161.092000,158.612000,161.092000,158.612000,"8,43B","1,56%"
9,01.12.2025,158.611000,159.073000,159.224000,158.029000,"7,50B","-0,29%"


### 2.4 - Visão Geral da Base de Dados

In [ ]:
# Quantidade de linhas e colunas do dataframe
print(f'Linhas: {df.shape[0]}')
print(f'Colunas: {df.shape[1]}')

Linhas: 502
Colunas: 7


In [ ]:
# Informações gerais da base de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      502 non-null    object 
 1   Último    502 non-null    float64
 2   Abertura  502 non-null    float64
 3   Máxima    502 non-null    float64
 4   Mínima    502 non-null    float64
 5   Vol.      502 non-null    object 
 6   Var%      502 non-null    object 
dtypes: float64(4), object(3)
memory usage: 27.6+ KB


- A coluna **Data** foi tratada como *object* pelo Pandas e precisa ser tratada: convertê-la e padronizá-la para o tipo *date*, além de ordená-la temporalmente (do mais antigo para o mais recente);
- Além disso, as colunas **Vol.** e **Var%** foram também tradadas como *object* pelo Pandas, em razão de caracteres indesejados como o **B** e **%**. Para o modelo isso é um problema e, por isso, essas colunas serão tratadas da devida maneira.

### 2.5 - Pré-processamento

In [ ]:
# Converter datas (dayfirst porque vem no formato: '24.11.2025', sendo o dia a primeira informação)
df["Data"] = pd.to_datetime(df["Data"], dayfirst=True)

# Ordenando temporalmente a base de dados, do mais antigo para o mais recente
df = df.sort_values("Data").reset_index(drop=True)

In [ ]:
# Visualizando o df ordenado temporalmente
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2023-12-13,129.465000,126.406000,129.793000,126.299000,"15,07M","2,42%"
1,2023-12-14,130.842000,129.469000,131.260000,129.469000,"15,82M","1,06%"
2,2023-12-15,130.197000,130.842000,131.661000,129.884000,"15,14M","-0,49%"
3,2023-12-18,131.084000,130.202000,131.447000,130.198000,"9,60M","0,68%"
4,2023-12-19,131.851000,131.088000,132.047000,131.086000,"9,43M","0,59%"


- Como foi falado, as colunas **Vol.** e **Var%** precisam ser tratadas e convertidas para o tipo correto, float.
- A coluna **Vol.** é peculiar pois apresenta sufixos que representam valores diferentes (B:Bilhões, M:Milhões, etc.). São, de fato, multiplicadores e precisam de um tratamento diferente.

In [ ]:
# Verificando os valores únicos da coluna Volume
df['Vol.'].unique()

array(['15,07M', '15,82M', '15,14M', '9,60M', '9,43M', '10,34M', '8,68M',
       '8,23M', '5,15M', '6,17M', '7,81M', '8,44M', '8,70M', '8,97M',
       '9,20M', '8,50M', '9,29M', '8,96M', '9,76M', '10,66M', '5,75M',
       '11,91M', '9,95M', '12,46M', '11,96M', '9,51M', '9,37M', '8,82M',
       '8,76M', '8,51M', '10,04M', '12,50M', '10,23M', '10,98M', '9,64M',
       '13,78M', '15,43M', '13,63M', '12,12M', '7,75M', '10,12M', '5,80M',
       '12,24M', '12,59M', '10,77M', '9,23M', '7,44M', '10,05M', '9,05M',
       '12,00M', '9,77M', '7,97M', '9,69M', '11,06M', '7,35M', '11,94M',
       '8,90M', '9,48M', '8,84M', '14,99M', '10,51M', '11,18M', '10,82M',
       '9,57M', '8,06M', '9,44M', '9,92M', '9,94M', '9,07M', '11,03M',
       '13,24M', '9,10M', '8,13M', '8,49M', '10,45M', '8,88M', '10,25M',
       '13,50M', '14,01M', '11,37M', '13,60M', '10,26M', '11,26M',
       '10,53M', '10,09M', '8,39M', '10,89M', '11,55M', '12,80M', '8,78M',
       '10,18M', '9,79M', '13,37M', '12,57M', '8,92M', '

- A príncipio, exitem apenas os multiplicadores **M** e **B**. Porém, a fim de evitar possíveis surpresas posteriormente, adicionaremos os multiplicadores **K** (Milhar) e **T** (Trilhões).

- Antes, vamos tratar os demais caracteres em todas as colunas da base de dados:

In [ ]:
# Criando uma função auxiliar para tratamento de caracteres
def remove_caracteres(valor):
  # Remove espaços
  valor.strip()

  # Se 'Valor' é instância (is instance) do tipo String
  if isinstance(valor, str):
    valor = valor.replace('.', '')
    valor = valor.replace('%', '')
    valor = valor.replace(',', '.')
  return valor

In [ ]:
# Aplicando a função auxiliar em todos as colunas da base de dados
for col in df.columns[1:]:
  df[col] = df[col].apply(remove_caracteres)

In [ ]:
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2023-12-13,129.465000,126.406000,129.793000,126.299000,15.07M,2.42
1,2023-12-14,130.842000,129.469000,131.260000,129.469000,15.82M,1.06
2,2023-12-15,130.197000,130.842000,131.661000,129.884000,15.14M,-0.49
3,2023-12-18,131.084000,130.202000,131.447000,130.198000,9.60M,0.68
4,2023-12-19,131.851000,131.088000,132.047000,131.086000,9.43M,0.59


- Como podemos ver, resta apenas tratar a coluna **Vol.**.

In [ ]:
# Criando uma função auxiliar para tratamento da coluna Volume
def tratamento_volume(volume):

  # Contornando o problema da existência de valores nulos
  if volume is None or (isinstance(volume, float)) and np.isnan(volume):
    return np.nan

  # Normalizando a string, padronizando para maiúsculo
  volume = volume.strip().upper()

  # Criando um dicionário com os multiplicadores
  multiplicadores = {"B": 1e9, "M": 1e6, "K": 1e3, "T": 1e12}

  # Detectando o multiplicador e subtituindo
  if volume[-1] in multiplicadores:
    return float(volume[:-1]) * multiplicadores[volume[-1]]
  else:
    return float(volume)

In [ ]:
# Aplicando a função auxiliar na coluna Volume
df["Vol."] = df["Vol."].apply(tratamento_volume)

In [ ]:
# Visualizando o resultado final:
df.head()

,Date,Close,Open,High,Low,Volume,Var%
0,2023-12-13,129.465000,126.406000,129.793000,126.299000,15070000.000000,2.42
1,2023-12-14,130.842000,129.469000,131.260000,129.469000,15820000.000000,1.06
2,2023-12-15,130.197000,130.842000,131.661000,129.884000,15140000.000000,-0.49
3,2023-12-18,131.084000,130.202000,131.447000,130.198000,9600000.000000,0.68
4,2023-12-19,131.851000,131.088000,132.047000,131.086000,9430000.000000,0.59


- Renomeando as colunas para o padrão Investing, com o intuito de facilitar o trabalho com a base de dados posteriormente.

In [ ]:
df = df.rename(columns={
    "Data": "Date",
    "Último": "Close",
    "Abertura": "Open",
    "Máxima": "High",
    "Mínima": "Low",
    "Vol.": "Volume",
    "Var%": "Return"
})

In [ ]:
df.head()

,Date,Close,Open,High,Low,Volume,Return
0,2023-12-13,129.465000,126.406000,129.793000,126.299000,15070000.000000,2.42
1,2023-12-14,130.842000,129.469000,131.260000,129.469000,15820000.000000,1.06
2,2023-12-15,130.197000,130.842000,131.661000,129.884000,15140000.000000,-0.49
3,2023-12-18,131.084000,130.202000,131.447000,130.198000,9600000.000000,0.68
4,2023-12-19,131.851000,131.088000,132.047000,131.086000,9430000.000000,0.59


- Por fim, convertendo **Volume** e **Return** para o tipo *float*.

In [ ]:
# Convertendo a tipagem das colunas
for col in df.columns[5:]:
  df[col] = df[col].astype(float)

In [ ]:
# Verificando a tipagem final da base de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    502 non-null    datetime64[ns]
 1   Close   502 non-null    float64       
 2   Open    502 non-null    float64       
 3   High    502 non-null    float64       
 4   Low     502 non-null    float64       
 5   Volume  502 non-null    float64       
 6   Return  502 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 27.6 KB


### 2.6 - Checando Valores Nulos e/ou Duplicados

In [ ]:
# Checando valores nulos
df.isnull().sum().sum()

np.int64(0)

- Não há valores nulos na base de dados.

In [ ]:
# Checando valores duplicados
df.duplicated().sum()

np.int64(0)

- Não há valores duplicados na base de dados.

## 3 - Análise Exploratória (EDA)  

## 4 - Engenharia de Atributos (Feature Engineering)

In [ ]:
# Ainda não entendi muito bem as features criadas aqui e acho que precisam ser melhor justificadas:
  # Com qual intuito elas foram criadas?
  # Como elas descrevem o comportamento da base com relação ao tempo?
  # Como elas influenciam os modelos a ter uma maior acertividade?

O fechamento de amanhã será maior que o de hoje?

Resultado:

1 → Alta

0 → Baixa

cria a variável alvo (Target) que o modelo vai aprender

df["Close"] É o preço de fechamento do dia atual.
df["Close"].shift(-1) df["Close"].shift(-1) e com isso, O fechamento de amanhã passa a ficar na linha de hoje.

(df["Close"].shift(-1) > df["Close"]) Se amanhã > hoje = True

Caso contrário = False

.astype(int) Transforma:

True = 1

False = 0

In [ ]:
df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)


Remover a última linha (não tem amanhã) pois amanhã é sábado

In [ ]:
df = df.dropna().reset_index(drop=True)


Conferir balanceamento das classes

Mercado tende a ~50% / 50%

Pequeno desbalanceamento é normal

df["Target"]

Seleciona a coluna Target, que criamos assim:

1 = Alta

0 = Baixa

.value_counts()

Conta quantas vezes cada valor aparece.

Exemplo:

1 = 120 vezes
0 = 100 vezes

normalize=True

Transforma as contagens em proporções (percentuais).

Modelos de classificação funcionam melhor quando:

As classes não estão muito desbalanceadas

In [ ]:
df["Target"].value_counts(normalize=True)


A variável alvo apresentou distribuição relativamente balanceada, não sendo necessária a aplicação de técnicas de balanceamento de classes.

Retorno diário (Importante!) Ela cria uma nova coluna chamada Return, que representa o retorno percentual diário do IBOVESPA.

Quanto o índice variou, em porcentagem, de um dia para o outro.

.pct_change()

Significa “percentual de mudança” entre uma linha e a anterior.

Como a árvore usa essa feature?

A árvore aprende regras como:

“Se o retorno de ontem foi maior que 0,3%, então a chance de alta amanhã aumenta.”

Sem retorno, isso seria muito mais difícil usando apenas preço.

In [ ]:
# Já existia um "Return", não?
df["Return"] = df["Close"].pct_change()


memória do passado

Retorno de ontem

Retorno de 2 dias atrás

Retorno de 3 dias atrás

shift(n) desloca os valores para baixo em n linhas.

Retorno original
Dia	Return
Dia 1	NaN
Dia 2	0.006
Dia 3	-0.004
Dia 4	0.010

Por que surgem NaN?

Nos primeiros dias:

Não existe retorno de 1, 2 ou 3 dias atrás

Então o valor fica NaN

pois peguei poucos dias para teste

In [ ]:
df["Return_lag1"] = df["Return"].shift(1)
df["Return_lag2"] = df["Return"].shift(2)
df["Return_lag3"] = df["Return"].shift(3)


In [ ]:
df.head(10)

Médias móveis (tendência)

O que são médias móveis?

Uma média móvel é uma média simples de um conjunto de valores ao longo de um período de tempo. Ela é calculada a cada novo dia, levando em conta o número de dias definidos.

No seu caso, temos:

MM5 → Média dos últimos 5 dias de fechamento

MM10 → Média dos últimos 10 dias

MM20 → Média dos últimos 20 dias

Essas médias são usadas para capturar tendências do mercado, suavizando flutuações diárias.

Como isso funciona no código?
.rolling(window=n)

A função .rolling(window=n) cria uma "janela deslizante" de n dias sobre a série de dados, ou seja, ela move a janela para frente dia a dia.

.mean()

Depois, a função .mean() calcula a média dos últimos n dias dentro dessa janela.

MM5 (Média dos últimos 5 dias):

Para o Dia 5, a média dos últimos 5 fechamentos será:

MM5
=
159.000
+
160.000
+
161.000
+
158.000
+
157.000 /
5
=
159.00

E assim por diante. À medida que a janela "anda", a média muda.

In [ ]:
df["MM5"] = df["Close"].rolling(window=5).mean()
df["MM10"] = df["Close"].rolling(window=10).mean()
df["MM20"] = df["Close"].rolling(window=20).mean()


Volatilidade (risco)

O que essa linha cria?

Ela cria uma nova coluna chamada Vol_5, que representa a volatilidade dos retornos nos últimos 5 dias.

O quanto o mercado tem oscilado recentemente nos 5 dias

In [ ]:
df["Vol_5"] = df["Return"].rolling(window=5).std()


## 5 - Definição do Target e Preparação da Base

## 6 - Separação da Base e Janela de Previsão (Últimos 30 dias)

## 7 - Modelos e Justificativa

## 8 - Resultados e Métricas de Avaliação

## 9 - Análise, Limitações e Melhorias